# Hloubkové senzory - detekce kolizí
Cvičení je zaměřené na práci s [hloubkovými mapami](https://en.wikipedia.org/wiki/Depth_map) vzniklých použitím tzv. hloubkových senzorů. V laboratoři jsou k dispozici hloubkové senzory založené na několika principech zisku hloubkových map. Jedná se o kamery typu Kinect, RealSense nebo Time-of-Flight (ToF). 

Vytvořit stereokameru lze i pomocí dvou obyčejných kamer. Podrobný návod pro zájemce je k dispozici na příklad [zde](https://erget.wordpress.com/2014/02/01/calibrating-a-stereo-camera-with-opencv/).

## Typy systémů
<table>
    <tr>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Kinect">Microsoft Kinect</a></th>
        <th align="left" width="16%"><img src="images/kinect.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Intel_RealSense">Intel RealSense</a></th>
        <th align="left" width="16%"><img src="images/realsense.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Time-of-flight_camera">Basler ToF</a></th>
        <th align="left" width="16%"><img src="images/tof.png" width="50%"/></th>       
    </tr>
    <tr>
        <td align="left" colspan="2">Postaven na principu promítání laserového vzoru a měření jeho deformace infračervenou kamerou.</td>
        <td align="left" colspan="2">Využívá stejného principu jako Kinect, pouze má jiný vzor.</td>
        <td align="left" colspan="2">Využívá promítání světla z více LED zdrojů a měří čas jeho návratu (odrazu).</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
        <td align="left" colspan="2">Měl by být vhodný i pro měření venku.</td>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Více Kinectů vedle sebe si vzájemně kazí vzory.</td>
        <td align="left" colspan="2">Nejnovější a nejmenší senzor na trhu.</td>
        <td align="left" colspan="2">Průmyslová konstrukce pro běh 24/7.</td>
    </tr>
</table>

### Import knihoven a konfigurace
Lze využívat i všechny funkce z minulých cvičení. Funkce byly odděleny do samostatného notebooku [svz.ipynb](../svz.ipynb). Pokud klik na odkaz hodí `Error 404`, znamená to, že jupyter notebook nemá přístup k tomuto notebooku. Je třeba spustit jupyter notebook znovu s adresou rootovské složky na adrese minimálně `bi-svz/tutorials/files` (nebo i výše).

In [1]:
%run ../svz.ipynb

from ipywidgets import interact_manual
import ipywidgets as widgets

import pyrealsense2 as rs
import datetime
import random
from enum import Enum, unique

### Pomocné třídy a funkce

In [2]:
def vizualize_depth_map(depth, threshold_near=0, threshold_far=10, inpaint=False):
    # Filtration in depth
    depth[depth > threshold_far] = 0
    depth[depth < threshold_near] = threshold_near
    
    # Min-max normalization - v’ = (v-min)/(max-min) * (newmax-newmin) + newmin
    depth_scaled = (((depth - depth.min()) / (depth.max() - depth.min() + 0.01)) * 255).astype(np.uint8)

    # Inpaint zero values with values from neighbourhood
    if inpaint:
        _, mask = cv2.threshold(depth_scaled, 1, 255, cv2.THRESH_BINARY_INV)
        depth_scaled = cv2.inpaint(depth_scaled, mask, 3, cv2.INPAINT_TELEA)

    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    depth_colormap = cv2.applyColorMap(depth_scaled, cv2.COLORMAP_JET)
    # FIXME: This works only on JET palette, because of [128,0,0]
    depth_colormap[np.where((depth_colormap==[128,0,0]).all(axis=2))] = [0,0,0]
    return depth_colormap

In [3]:
@unique
class ImageType(Enum):
    RGB = 1
    DEPTH = 2
    DEPTH_RAW = 3
    COMBI = 4

class RealSenseCVViewer:
    WIDGET_TYPES = {
        'int': widgets.IntSlider,
        'float': widgets.FloatSlider,
        'bool': widgets.Checkbox,
        'int_text': widgets.BoundedIntText,
        'float_text': widgets.BoundedFloatText
    }

    SUPPORTED_CONFIG_NAMES = [
        'Gain',
        'ExposureTime'
    ]
    
    def __init__(self, serial_number, depth_size=(1280, 720), rgb_size=(1280, 720), enable_depth=True, enable_color=True):
        # Device config
        self._serial_number = serial_number
        self._enable_depth = enable_depth
        self._enable_color = enable_color
        self._depth_frame_width = depth_size[0]
        self._depth_frame_height = depth_size[1]
        self._rgb_frame_width = rgb_size[0]
        self._rgb_frame_height = rgb_size[1]
        self._config = None
        self.ExposureTime = 800
        self.Gain = 39
        # Widget config
        self._interact_widgets = {}
        self._impro_function = None
        # Init device config
        self.set_camera(serial_number, enable_depth, enable_color)

    def set_camera(self, serial_number, enable_depth=True, enable_color=True):
        if not self._device_available():
            raise ValueError("RealSense device {} is not available.".format(self._serial_number))
        self._serial_number = serial_number
        self._enable_depth = enable_depth
        self._enable_color = enable_color
        self._config = rs.config()
        self._config.enable_stream(rs.stream.depth, self._depth_frame_width, self._depth_frame_height, rs.format.z16, 30)
        self._config.enable_stream(rs.stream.color, self._rgb_frame_width, self._rgb_frame_height, rs.format.bgr8, 30)
        self._config.enable_device(self._serial_number)

    def set_features(self, features):
        new_interact_widgets = {}

        for feature in features:
            widget_kwargs = {}

            if not isinstance(feature, dict):
                raise ValueError("Feature is not dict type")
            # Extract name for widget
            feature_name = feature.get('name')
            if feature_name is None:
                raise ValueError("'name' attribute can't be None")
            # Is this feature implemented?
            if feature_name not in self.SUPPORTED_CONFIG_NAMES:
                raise ValueError("Feature {} is not supported yet.".format(feature_name))
            realsense_feature_default = self._get_default_feature_setting(feature_name)

            # realsense_feature = getattr(self._camera, feature_name)
            # Extract description for widget
            widget_kwargs['description'] = re.sub('([a-z])([A-Z])', r'\1 \2', feature_name) + " :"
            # Extract type
            type_name = feature.get('type', 'int')
            widget_obj = self.WIDGET_TYPES.get(type_name)
            if widget_obj is None:
                raise ValueError("Widget type name '{}' is not valid.".format(type_name))
            # Extract value
            value = feature.get('value', realsense_feature_default['value'])
            widget_kwargs['value'] = value

            if type_name != 'bool':
                step = feature.get('step')
                if step is None:
                    try:
                        step = realsense_feature_default['step']
                    except:
                        step = 1
                widget_kwargs['step'] = step

                max_value = feature.get('max', realsense_feature_default['max'])
                max_value = (realsense_feature_default['max'], max_value)[max_value <= realsense_feature_default['max']]
                widget_kwargs['max'] = max_value

                min_value = feature.get('min', realsense_feature_default['min'])
                min_value = (realsense_feature_default['min'], min_value)[min_value <= realsense_feature_default['min']]
                widget_kwargs['min'] = min_value

                layout = feature.get('layout', widgets.Layout(width='99%', height='50px'))
                widget_kwargs['layout'] = layout
            style = {'description_width': 'initial'}
            new_interact_widgets[feature_name] = widget_obj(style=style, **widget_kwargs)

        self._interact_widgets = new_interact_widgets

    def set_impro_function(self, impro_function, params_dict):
        if impro_function is not None and not callable(impro_function):
            raise ValueError("Object {} is not callable.".format(impro_function))
        self._impro_function = impro_function
        self._impro_function_params = params_dict
        
    def run_interaction_single_shot(self, window_size=None, image_folder='.'):
        if not self._device_available():
            raise ValueError("RealSense device {} is not available.".format(self._serial_number))
        if window_size is not None and not len(window_size) == 2:
            raise ValueError("Argum"
                             "ent 'window_size' has to be None or tuple of length 2.")
        interact_manual(self._single_interaction_function_wrap(window_size, image_folder),
                        **self._interact_widgets)

    def run_interaction_continuous_shot(self, window_size=None, image_folder='.'):
        if not self._device_available():
            raise ValueError("RealSense device {} is not available.".format(self._serial_number))
        if window_size is not None and not len(window_size) == 2:
            raise ValueError("Argument 'window_size' has to be None or tuple of length 2.")
        self._start_grabbing()
        interact_manual(self._continuous_interaction_function_wrap(window_size, image_folder),
                        **self._interact_widgets)

    def _single_interaction_function_wrap(self, window_size=None, image_folder='.'):
        def camera_configuration(**kwargs):
            # Refresh settings from GUI
            for feature, value in kwargs.items():
                setattr(self, feature, value)
            # Create simple window if no function is defined
            if self._impro_function is None:
                cv2.namedWindow('RealSense capture', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
                if window_size is not None:
                    cv2.resizeWindow('RealSense capture', window_size[0], window_size[1])
            # Grab one image
            self._start_grabbing()
            rgb, depth = self._grab_image()
            self._stop_grabbing()
            if self._impro_function is not None:
                rgb, depth = self._impro_function(rgb, depth, **self._impro_function_params)
            else:
                # Stack both images horizontally
                cv2.imshow('RealSense capture', np.hstack(rgb, vizualize_depth_map(depth)))
            
            # Wait for user's action
            while True:
                k = cv2.waitKey(1) & 0xFF
                if k == ord('s'):
                    cv2.imwrite(os.path.join(image_folder, 'RealSenseRgbImage-' + str(
                        int(datetime.datetime.now().timestamp())) + '.png'), rgb)
                elif k == ord('d'):
                    cv2.imwrite(os.path.join(image_folder, 'RealSenseDepthImage-' + str(
                    int(datetime.datetime.now().timestamp())) + '.png'), vizualize_depth_map(depth))
                elif k == ord('q'):
                    break
            # Dispose
            cv2.destroyAllWindows()
            return
        return camera_configuration

    def _continuous_interaction_function_wrap(self, window_size=None, image_folder='.'):
        def camera_configuration(**kwargs):
            for feature, value in kwargs.items():
                setattr(self, feature, value)
            if self._impro_function is None:
                cv2.namedWindow('RealSense capture', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
                if window_size is not None:
                    cv2.resizeWindow('RealSense capture', window_size[0], window_size[1])
            self._start_grabbing()
            
            while True:
                rgb, depth = self._grab_image()
                if self._impro_function is not None:
                    rgb, depth = self._impro_function(rgb, depth, **self._impro_function_params)
                else:
                    cv2.imshow('RealSense capture', np.hstack(rgb, vizualize_depth_map(depth)))
                
                k = cv2.waitKey(1) & 0xFF
                if k == ord('s'):
                    cv2.imwrite(os.path.join(image_folder, 'RealSenseRgbImage-' + str(
                        int(datetime.datetime.now().timestamp())) + '.png'), rgb)
                elif k == ord('d'):
                    cv2.imwrite(os.path.join(image_folder, 'RealSenseDepthImage-' + str(
                    int(datetime.datetime.now().timestamp())) + '.png'), vizualize_depth_map(depth))
                elif k == ord('q'):
                    break
            cv2.destroyAllWindows()
            self._stop_grabbing()
            return
        return camera_configuration

    def save_image(self, filename):
        if not self._device_available():
            raise ValueError("RealSense device {} is not available.".format(self._serial_number))

    def get_image(self, image_type=ImageType.DEPTH):
        if not self._device_available():
            raise ValueError("RealSense device {} is not available.".format(self._serial_number))
        self._start_grabbing()
        rgb, depth = self._grab_image()
        self._stop_grabbing()
        
        if image_type is ImageType.DEPTH:
            return vizualize_depth_map(depth)
        elif image_type is ImageType.RGB:
            return rgb
        elif image_type is ImageType.COMBI:
            return np.hstack(rgb, vizualize_depth_map(depth))
        elif image_type is ImageType.DEPTH_RAW:
            return depth
        else:
            return None

    def _start_grabbing(self):
        self._pipeline = rs.pipeline()
        self._profile = self._pipeline.start(self._config)
        # Set gain end exposure time
        self._profile.get_device().query_sensors()[1].set_option(rs.option.exposure, self.ExposureTime)
        self._profile.get_device().query_sensors()[1].set_option(rs.option.gain, self.Gain)
        
        # Depth scale - units of the values inside a depth frame, i.e how to convert the value to units of 1 meter
        depth_sensor = self._profile.get_device().first_depth_sensor()
        self._depth_scale = depth_sensor.get_depth_scale()
        
    def _stop_grabbing(self):
        self._pipeline.stop()
        
    def _grab_image(self):
        # Get images
        frames = self._pipeline.wait_for_frames(5000)
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Converts to meters
        depth_meters = depth_image * self._depth_scale
        
        return color_image, depth_meters

    def _device_available(self):
        context = rs.context()
        devices = context.query_devices()
        for device in devices:
            serial_number = device.get_info(rs.camera_info.serial_number)
            if serial_number == self._serial_number:
                return True
        return False

    def _get_default_feature_setting(self, feature_name):
        if feature_name not in self.SUPPORTED_CONFIG_NAMES:
            raise ValueError("Feature {} is not supported yet.".format(feature_name))
        if feature_name == 'Gain':
            return {'name': 'Gain',
                    'description': 'Gain',
                    'type': 'float',
                    'value': 39.0,
                    'min': 0.0,
                    'max': 128.0,
                    'step': 1.0}
        elif feature_name == 'ExposureTime':
            return {'name': 'ExposureTime',
                    'description': 'Exposure time',
                    'type': 'float',
                    'value': 800.0,
                    'min': 1.0,
                    'max': 10000,
                    'step': 1.0}

features = [
    {
        "name": "Gain",
        "description": "Gain",
        "type": "float",
        "value": 39.0,
        "min": 0.0,
        "max": 128.0,
        "step": 1.0
    },
    {
        "name": "ExposureTime",
        "description": "Exposure time",
        "type": "float",
        "value": 800.0,
        "min": 1.0,
        "max": 10000,
        "step": 1.0
    }
]

In [4]:
class EnemyTarget:
       
    def __init__(self, depth_from, depth_to, img_size):
        self.depth_from = depth_from
        self.depth_to = depth_to
        self.img_size = img_size
        self.dead_count = 0
        self.coll_toler_depth = 0.05
        self.coll_toler_coords = 50
        self.is_alive = False
        self.coords = None
        self.depth = None

        self.generate_new()
        
    def generate_new(self):
        self.depth = random.uniform(self.depth_from, self.depth_to)
        magic_constant = 2/3
        # Pure magic
        self.coords = (int(self.img_size[0]/2 + random.uniform(-magic_constant * self.img_size[0]/2 , magic_constant * self.img_size[0]/2)),
                         int(self.img_size[1]/2 + random.uniform(-magic_constant* self.img_size[1]/2 , magic_constant * self.img_size[1]/2)))
        self.is_alive = True

    def check_collision(self, sq_rect, sq_depth):
        if sq_rect is None or self.is_alive is False:
            return False
        sq_x, sq_y = sq_rect[0]
        if (abs(sq_x - self.coords[0]) < self.coll_toler_coords 
                and abs(sq_y - self.coords[1]) < self.coll_toler_coords
                and abs(sq_depth - self.depth) < self.coll_toler_depth):
            return True
        
        return False
    
    def collide(self):
        self.is_alive = False
        self.coords = None
        self.depth = None
        self.dead_count += 1
        print('Získán bod. Celkový počet bodů je: ' + str(self.dead_count))

def collision_detect(rgb, depth, **kwargs):
    depth = cv2.rotate(depth, cv2.ROTATE_180)
    sq_cont, sq_rect, sq_depth = find_square(depth, **kwargs)
    target = kwargs['enemy_target']
    collides = target.check_collision(sq_rect, sq_depth)
    
    if collides:
        target.collide()
        target.generate_new()
    
    depth[target.coords[1] - 5 : target.coords[1] + 5, target.coords[0] - 5 : target.coords[0] + 5] = target.depth
    
    sq_target_drawn = cv2.rectangle(np.zeros(depth.shape, dtype=np.uint8), (target.coords[0] - 5, target.coords[1] - 5), 
                                (target.coords[0] + 5 , target.coords[1] + 5), 
                                color=(255, 255, 255), thickness=cv2.FILLED)
    if sq_cont is not None:
        sq_target_drawn = cv2.drawContours(sq_target_drawn, [sq_cont], -1, color=(255, 255, 255), thickness=cv2.FILLED)
    
    depth_visualized = vizualize_depth_map(depth, kwargs['thresh_near'], kwargs['thresh_far'])
    depth_masked = cv2.bitwise_and(depth_visualized, depth_visualized, mask=sq_target_drawn)
    show_camera_window(depth_masked)
    
    return rgb, depth

In [5]:
def area_select(rgb, depth, **kwargs):
    show_camera_window(vizualize_depth_map(depth, threshold_far=kwargs['thresh_far'], threshold_near=kwargs['thresh_near']))
    return rgb, depth

def show_scale(min_val, max_val, color_map='jet'):
    fig = plt.figure(figsize=(8, 3))
    ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
    norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
    cb1 = matplotlib.colorbar.ColorbarBase(ax1, cmap=plt.cm.jet, norm=norm, orientation='horizontal')
    cb1.set_label('Depth (meters)')
    plt.show()

---

### Úkol
Cílem cvičení je využít hloubkovou mapu ze senzoru RealSense pro detekci kolizí objektů. K tomu je zapotřebí být schopen počítat reálné rozměry objektů v hloubkových mapách.

K výpočtu reálných rozměrů objektů v hloubkové mapě je zapotřebí znát typ senzoru, se kterým se pracuje a jeho zorný úhel jak v horizontálním tak ve vertikálním směru (tzv. FOV). Využívá se geometrie viz následující obrázek (FOV z obrázku **NENÍ** naše hledané). 

![](https://i.stack.imgur.com/Wf3bu.png)

#### 1) Najděte správné hodnoty FOV pro váš senzor a doplňte.

In [ ]:
FOV_HORIZONTAL_DEGREES =  ###
FOV_VERTICAL_DEGREES =  ###

def real_measure(w_px, h_px, depth, frame_width, frame_height):
    """
    Function for counting the real measures in meters out of pixel values in depth map images.
    Source: https://stackoverflow.com/a/45481222/1398955
    
    @param w_px : int
        Width in pixels.
    @param h_px : int
        Height in pixels.
    @param depth : float
        Depth value in meters.
    @param frame_width : int
        Frame width in pixels.
    @param frame_height : int
        Frame height in pixels.
    @return w_m : int
        Width in meters.
    @return h_m : int
        Height in meters.
    """
    fov_horizontal = FOV_HORIZONTAL_DEGREES * np.pi / 180.0
    fov_vertical = FOV_VERTICAL_DEGREES * np.pi / 180.0

    horizontal_scaling = 2 * np.tan(fov_horizontal / 2.0) / float(frame_width)
    vertical_scaling = 2 * np.tan(fov_vertical / 2.0) / float(frame_height)

    w_m = w_px * horizontal_scaling * depth
    h_m = h_px * vertical_scaling * depth
    
    return w_m, h_m

#### 2) Doplňte reálné hodnoty nastavení scény
Doplňte sériové číslo senzoru, reálné rozměry měřené scény a reálné rozměry snímaného objektu. Seznamte se s barevnou škálou pro určení hloubky objektů.

In [7]:
serial_number = '' ###
size = (?, ?) ###

# Scene specific measures (meters)
thresh_near =  ###
thresh_far =  ###
thresh_step =  ###

# Object specific measures (meters)
object_w_m =  ###
object_h_m =  ###
tolerance =  ###

# Shows distance using colors scale bar
show_scale(thresh_near, thresh_far)

SyntaxError: invalid syntax (<ipython-input-7-a89f0bd05dd2>, line 2)

In [ ]:
# Settings dictionary creation - DO NOT modify
params_dict = dict()
params_dict['image_size'] = size
# Depth map thresholds in meters
params_dict.update({'thresh_far': thresh_far, 'thresh_near': thresh_near, 'thresh_step': thresh_step})
# Real object sizes and measurement tolerance
params_dict.update({'object_w_m': object_w_m, 'object_h_m': object_h_m, 'tolerance': tolerance})
params_dict['enemy_target'] = EnemyTarget(params_dict['thresh_near'], params_dict['thresh_far'], params_dict['image_size'])

#### 3) Spusťte snímání a vyzkoušejte si, jak vypadá hloubková mapa
Seznamte se s použitím barevné škály v reálném obraze pro určení hloubky objektů.

In [6]:
tof_stream(area_select, params_dict)

NameError: name 'params_dict' is not defined

In [ ]:
viewer = RealSenseCVViewer(serial_number, depth_size=params_dict['image_size'], rgb_size=params_dict['image_size'])
viewer.set_features(features)
viewer.set_impro_function(area_select, params_dict)
viewer.run_interaction_continuous_shot()

#### 4) Doplňte funkci pro nalezení objektu v obraze
Je zapotřebí najít vždy pouze objekt a ne jeho držák nebo jiné falešně pozitivní objekty popř. šum. 

**Pozn.:** _Znak `\` uprostřed zdrojového kódu v Pythonu umožňuje rozdělit např. podmínku na více řádků._

In [ ]:
def find_square(depth, **kwargs):
    thresh_near = kwargs['thresh_near']
    thresh_far = kwargs['thresh_far']
    thresh_step = kwargs['thresh_step']
    object_w_m = kwargs['object_w_m']
    object_h_m = kwargs['object_h_m']
    tolerance = kwargs['tolerance']
    
    # For each depth plane
    for i in np.arange(thresh_near, thresh_far-thresh_step, thresh_step):
        # Clones depth data in one plane to get rid of object holder
        depth_clone = np.zeros(depth.shape)
        locs = np.where((depth > i) & (depth < i + thresh_step))       
        depth_clone[locs[0], locs[1]] = depth[locs[0], locs[1]] * 42 # Just shift by magic constant (basically any number greater than 10 is acceptable)
        depth_clone = depth_clone.astype(np.uint8)
    
        conts, _ = cv2.findContours(depth_clone, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        #FIXME: Sorry for magic constants but it works and speeds up the process ;)
        for c in [c for c in conts if cv2.contourArea(c) > 1500 and cv2.contourArea(c) < 15000]:
            rect = cv2.minAreaRect(c)
            c_x, c_y = rect[0]
            w_px, h_px = rect[1]
            rect_area = w_px * h_px
            square_depth = ... # depth value of found contour
            
            ### Zvolte správnou funkci se správnými parametry pro měření rozměrů
            w_m, h_m =  ###

            ### Vytvořte if, ve kterém specifikujete 3 podmínky pro nalezení objektu
            # ... velikostí odpovídá skutečnosti
            # ... je to čtverec (podle rozměrů)
            # ... je to vyplněný čtverec (podle obsahu)
            if ... \
                    and ... \
                    and ... :
                return c, rect, square_depth
         
    return None, None, None

#### 5) Spusťte snímání a staňte se robotickou rukou
Dostaňte kostku do cílového místa a získejte co nejvíc bodů.

In [ ]:
tof_stream(collision_detect, params_dict)

In [ ]:
viewer = RealSenseCVViewer(serial_number, depth_size=params_dict['image_size'], rgb_size=params_dict['image_size'])
viewer.set_features(features)
viewer.set_impro_function(collision_detect, params_dict)
viewer.run_interaction_continuous_shot()